In [9]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [10]:
import random, string, numpy
import time

def init(): 
    pop = []   
    for i in range(pop_size):
        genotype = ''.join(random.choice(string.ascii_letters) for x in range(genotype_size))
        pop.append(genotype)   
    return(pop)
    

In [11]:
def evaluate(pop):
    fitness = []
    for i in range(pop_size):
        genotype = pop[i]
        fitness_value = sum(ord(i) for i in genotype)  #sums up the ascii values of the characters
        fitness.append(fitness_value)    
    # Normalize fitness. IMPORTANT: If you don't multiple by 1.0, you will get all zeros
    fitness = [f * 1.0/sum(fitness) for f in fitness]
    # This normalization is needed for select() to pick the parents    
    return(fitness)

In [12]:
def select(fitness, pop):    
    # Select first parent
    parent_1_index = -9999  #indicates that parent_1 is yet to be chosen
    cumulated_fitness = 0
    roulette_marker = random.random()  #indicates that the 'roulette wheel' has settled on a random number between 0 and 1
    for i in range(pop_size):
        cumulated_fitness += fitness[i]
        if cumulated_fitness >= roulette_marker:
            parent_1_index = i
            break
    
    # Select second parent different from the first parent
    parent_2_index = parent_1_index  #indicates that parent_2 is yet to be chosen
    while parent_2_index == parent_1_index:  #this ensures that the two parents chosen are distinct
        cumulated_fitness = 0
        roulette_marker = random.random()  #indicates that the 'roulette wheel' has settled on a random number
        for i in range(pop_size):
            cumulated_fitness += fitness[i]
            if cumulated_fitness >= roulette_marker:
                parent_2_index = i
                break
    
    return([pop[parent_1_index], pop[parent_2_index]])

In [13]:
def recombine(parent_1, parent_2, recombination_rate):   
    r = random.random()   
    if r <= recombination_rate:  #recombination_rate is a value between 0 and 1
        #recombine
        slice_point = random.randint(0, genotype_size-1)
        offspring_1 = parent_1[0 : slice_point] + parent_2[slice_point : genotype_size]
        offspring_2 = parent_2[0 : slice_point] + parent_1[slice_point : genotype_size]
        return([offspring_1, offspring_2])
   
    else:  #don't recombine
        return([parent_1, parent_2])

In [14]:
def mutate(genotype, mutation_rate):   
    mutated_genotype = genotype  #indicates that the genotype is yet to be mutated   
    for i in range(genotype_size):   
        r = random.random()   
        if r <= mutation_rate:  #rate is a value between 0 and 1
            # then mutate this symbol by a small shift on the alphabet scale, else proceed to the next symbol            
            # 'numpy.random.poisson' draws a random integer from a bell shaped distribution centered at 'lam',
            # where 'lam' is short for 'lambda' that represents the mean of the distribution.
            # So 'numpy.random.poisson(lam = 2)' typically returns the number 2, occasionally 1 and 3, even more
            # rarely 0 and 4.            
            mutation_magnitude = numpy.random.poisson(lam = 2)  
            mutation_direction = random.choice([-1, 1])
            mutation = mutation_direction * mutation_magnitude            
            present_symbol_ascii = ord(genotype[i])            
            mutated_symbol_ascii = present_symbol_ascii + mutation            
            if (mutated_symbol_ascii < ord('A')): mutated_symbol_ascii = ord('A')
            if (mutated_symbol_ascii > ord('z')): mutated_symbol_ascii = ord('z')
            new_symbol = chr(mutated_symbol_ascii)            
            mutated_genotype = mutated_genotype[0 : i] + new_symbol + mutated_genotype[(i+1) : genotype_size]
    return(mutated_genotype)

Revise the functions above and the code below to implement a Genetic Algorithm to evolve a 'textual' genotype in the following way.

- Your init() method should initialize each genotype in the population to consist of a string of random letters. 
- The objective is to evolve genotypes that contain as many (lowercase) a's as possible. 
- That is, you can use the number of lowercase a's present in a genotype as its fitness score in your evaluate() method. 
- At the end of each generation, print out the best performing genotype.

For solving this simple problem (by GA standards), you may choose to not use the recombine() method, but you are welcome to try it.

Use a population size, genome size, and number of iterations of your preference.

NOTE: Your mutation method should not explicitly replace a genotype letter with 'a'. Although it will lead to solutions more quickly, it's not natural from an evolution point of view that is blind in a sense.

In [15]:


# Python program to illustrate ascii()
test_str = string.ascii_letters
print(ascii(test_str))

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'


In [16]:
def countLowerLetter(string):
    return sum(1 for char in string if char.islower())



In [39]:

# store starting time
beginTime = time.time()
# EXAMPLE GA parameers
pop_size = 20  #population size
num_generations = 10000
genotype_size = 10
recombination_rate = 0.2
mutation_rate =  0.2

# New stuff
maxAcceptedPop = []
maxCount = 0
setNumberOfLowercase = genotype_size - 3 #Set selected genotype
setNumberOfAcceptedGenoType = pop_size - 5 #Set condition for termination - selected genotype less than or equal this number
iterationCount = 0

print('COMP 670: GA Assignment by: Nghia Van') #Print your name

# Main GA loop.
# This is NOT the full GA. A full GA may do other things in between the lines below.
pop = init()


print("The initial population is: ", pop) #printing the initial population
for gen in range(num_generations):
	iterationCount = iterationCount + 1
	fitness = evaluate(pop)
	#print("Fitness: " + str(fitness))

	# create a new population that will hold the next generation of genotypes
	# if you are using 'elite' selection, then simply copy the top x% of individuals from pop to new_pop
	new_pop = []  

	while (len(new_pop) < pop_size):  # continue loop until new_pop has pop_size number of individuals
		[parent_1, parent_2] = select(fitness, pop)
		[offspring_1, offspring_2] = recombine(parent_1, parent_2, recombination_rate)
		mutated_genotype_1 = mutate(offspring_1, mutation_rate)
		mutated_genotype_2 = mutate(offspring_2, mutation_rate)
		
		new_pop.append(mutated_genotype_1)
		new_pop.append(mutated_genotype_2)
		
		# continue loop until new_pop has pop_size number of individuals
	pop = new_pop  #replace current population with new population
	numberOfAcceptGenoType = 0
	for i in range (pop_size):		
		numberOfLowerLetter = (countLowerLetter(pop[i]))
		if numberOfLowerLetter >= setNumberOfLowercase:
			numberOfAcceptGenoType = numberOfAcceptGenoType + 1
		
	if numberOfAcceptGenoType >= maxCount: #Try to pick the latest if it run all iterations
		maxCount = numberOfAcceptGenoType
		maxAcceptedPop = pop
		if maxCount >= setNumberOfAcceptedGenoType: #remove this 2 line of code to get the best pupulation in all iterations
			break #break to reach the termination setting; remove this code to get the best pupulation in all iterations

	#print("The current population is:   ", pop)

	# print best genotype and its fitness score of current generation
 	
	# continue loop to create next generation
# print overall best genotype and its fitness score at the end of num_generations

# store end time
endTime = time.time()
# total time taken
print("Iteration Count: " + str(iterationCount))
print(f"Total runtime of the program is {endTime - beginTime}")
print("The last population is   	         :  ", pop)
print("The best performing genotype population is: ", maxAcceptedPop)

COMP 670: GA Assignment by: Nghia Van
The initial population is:  ['LuFUkhnUfz', 'vtSiaToRkc', 'XcaYUACNJk', 'tIMvPwIITF', 'vpHoxkpPve', 'LitbGuzKWI', 'jgNzbOyVCh', 'ULzvthAroI', 'qVORLeMvmo', 'oveFHoVkPn', 'WbgjKUXXSy', 'GRokmoMiJY', 'MWALDfJMxx', 'akYLdxpViu', 'LnnNdnwERn', 'oseBJtTYbQ', 'nhcOoulzeX', 'OJAAAdMnuE', 'knFIfNMFuV', 'mYtllXNeXd']
Iteration Count: 458
Total runtime of the program is 0.10172891616821289
The last population is   	         :   ['Ac[cvnLnqr', 'JcPjwgCowy', 'Ac_fwjMrso', 'BfJjxgFpwu', 'J`U]zkNrqr', 'EcYjwjPzyp', 'ChOjriEqot', 'GcHiyfKptt', 'CBP^ujGuoy', 'BaJjzlHpuw', 'Ac[gwjJrqo', 'AcYjwjMrqr', 'E`SZzkMxwy', 'H`U]zkNzzp', 'JcPjxgEpxp', 'JcQjunFotz', 'CBP`wjGuoy', 'GeHiyfGosv', 'KcPjunDowp', 'JeHhyfGovx']
The best performing genotype population is:  ['Ac[cvnLnqr', 'JcPjwgCowy', 'Ac_fwjMrso', 'BfJjxgFpwu', 'J`U]zkNrqr', 'EcYjwjPzyp', 'ChOjriEqot', 'GcHiyfKptt', 'CBP^ujGuoy', 'BaJjzlHpuw', 'Ac[gwjJrqo', 'AcYjwjMrqr', 'E`SZzkMxwy', 'H`U]zkNzzp', 'JcPjxgEpxp', 'JcQ

In [18]:
# print overall best genotype and its fitness score at the end of num_generations
print("The finalinitial population is: ", pop)

The finalinitial population is:  ['nfppsYaUuk', 'qaktt_k`nr', 'oaspqci^ws', 'iesnrYbVuy', 'iesmsYbUuk', 'nhppoYbVus', 'ranpvX]^wu', 'oaqpjZg`sr', 'rarpvY]`wu', 'a^wpl\\eezz', 'mfrhpYdZpy', 'g^utmYf_pr', 'g^upmYfapp', 'g^upkYfaqq', 'n`kovak`nr', 'a^wpk\\eesz', 'oaootcn`qr', 'lbrtq]l`qt', 'a_vpmcg^wy', 'oasntYf`qr']
